In [1]:
import youtube_dl

In [10]:
# url = "https://www.youtube.com/watch?v=TI8nSJ9-C3c"
# ydl_opts = {
#     'format': 'bestaudio/best',
#     'ignoreerrors': True,
#     'outtmpl': 'electronic/%(title)s.%(ext)s',
#     'playlistend': 110,
#     'postprocessors': [{
#         'key': 'FFmpegExtractAudio',
# #         'preferredcodec': 'mp3',
#         'preferredquality': '192',
#     }],
# }
# with youtube_dl.YoutubeDL(ydl_opts) as ydl:
#     ydl.download(['https://www.youtube.com/watch?v=zXDAYlhdkyg&list=PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S'])

In [11]:
!ls electronic/

Blake Shelton - 'Hell Right (ft. Trace Adkins)' [Official Music Video].opus
Blake Shelton - God's Country (Official Music Video).opus
Brett Young - In Case You Didn't Know (Official Music Video).opus
Cole Swindell - 'Love You Too Late' (Official Music Video).opus
Dan + Shay - All To Myself (Shadow Video).opus
Gabby Barrett - I Hope (Official Video).m4a
Kane Brown - Heaven (Official Music Video).opus


In [2]:
youtube_dl.YoutubeDL?

Init signature: youtube_dl.YoutubeDL(params=None, auto_init=True)
Docstring:     
YoutubeDL class.

YoutubeDL objects are the ones responsible of downloading the
actual video file and writing it to disk if the user has requested
it, among some other tasks. In most cases there should be one per
program. As, given a video URL, the downloader doesn't know how to
extract all the needed information, task that InfoExtractors do, it
has to pass the URL to one of them.

For this, YoutubeDL objects have a method that allows
InfoExtractors to be registered in a given order. When it is passed
a URL, the YoutubeDL object handles it to the first InfoExtractor it
finds that reports being able to handle it. The InfoExtractor extracts
all the information about the video or videos the URL refers to, and
YoutubeDL process the extracted information, possibly using a File
Downloader to download the video.

YoutubeDL objects accept a lot of parameters. In order not to saturate
the object constructor with a